In [9]:
from diffusers import StableDiffusionInstructPix2PixPipeline,StableDiffusionPipeline
import torch
from h5reader import *
import random
import os
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image
from tqdm import tqdm
import json
from torch.utils.data import random_split
random.seed(42)

In [2]:
origin_pipe = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16, 
    safety_checker = None
).to("cuda")


model_path = "p2p_test_2"  
trained_pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  
    safety_checker = None
).to("cuda")

model_large = "p2p_test"

trained_large = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_large,
    torch_dtype=torch.float16,  
    safety_checker = None
).to("cuda")

Couldn't connect to the Hub: (ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 6a27ed5c-1c98-4c4f-a794-6170de03c294)').
Will try to load from local cache.
Loading pipeline components...: 100%|██████████| 6/6 [00:03<00:00,  1.64it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Loading pipeline components...: 100%|██████████| 6/6 [00:

In [11]:
dataset = FrameDataset(step = 30,begin = random.randint(0,30))
test_dataset,_ = random_split(dataset,[1000,len(dataset)-1000])
guidance_scale = 7.5
num_inference_steps = 50
image_guidance_scale = 1.5  


[....................................................................................................]


In [12]:
def load_image(img, size=(256, 256)):
    try:
        img = img.resize(size)
    except:
        pass
    img_np = np.array(img).astype(np.float32) / 255.0  # scale to [0,1]
    return img_np


def test_epoch(index):
    pair = test_dataset.__getitem__(index)
    prompt = pair['instruction']
    src = load_image(pair['src'])
    ground_truth = load_image(pair['tgt'])

    non_trained = origin_pipe(prompt = prompt,
                              image = src,
                              guidance_rescale= guidance_scale,
                              num_inference_steps=num_inference_steps,
                              
                              image_guidance_scale=image_guidance_scale).images[0]
    trained = trained_pipe(prompt = prompt,
                              image = src,
                              guidance_rescale= guidance_scale,
                              num_inference_steps=num_inference_steps,
                              image_guidance_scale=image_guidance_scale).images[0]
    
    large = trained_large(prompt = prompt,
                              image = src,
                              guidance_rescale= guidance_scale,
                              num_inference_steps=num_inference_steps,
                              image_guidance_scale=image_guidance_scale).images[0]
    record = {}
    
    record["idx"] = index
    record["prompt"] = prompt

    origin_img = load_image(non_trained)
    trained_img = load_image(trained)
    large_img = load_image(large)

    cv2.imwrite(f"test_imgs/begin_{index}.jpg",src*255)
    cv2.imwrite(f"test_imgs/gt_{index}.jpg",ground_truth*255)
    cv2.imwrite(f"test_imgs/origin_{index}.jpg",origin_img*255)
    cv2.imwrite(f"test_imgs/trained_{index}.jpg",trained_img*255)
    cv2.imwrite(f"test_imgs/large_{index}.jpg",large_img*255)

    ssim_val = ssim(ground_truth, origin_img, channel_axis=2, data_range=1.0)
    psnr_val = psnr(ground_truth, origin_img, data_range=1.0)
    record["origin_model"] = {"SSIM":float(ssim_val),"PSNR (dB)":float(psnr_val)}

    ssim_val = ssim(ground_truth, trained_img, channel_axis=2, data_range=1.0)
    psnr_val = psnr(ground_truth, trained_img, data_range=1.0)
    record["trained_model"] = {"SSIM":float(ssim_val),"PSNR (dB)":float(psnr_val)}

    ssim_val = ssim(ground_truth, large_img, channel_axis=2, data_range=1.0)
    psnr_val = psnr(ground_truth, large_img, data_range=1.0)
    record["large_model"] = {"SSIM":float(ssim_val),"PSNR (dB)":float(psnr_val)}

    return record
    
print(test_epoch(0))

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:02<00:00, 20.81it/s]


{'idx': 0, 'prompt': 'lay blocks one atop another                       ', 'origin_model': {'SSIM': 0.15910577774047852, 'PSNR (dB)': 5.136190219186173}, 'trained_model': {'SSIM': 0.511362612247467, 'PSNR (dB)': 10.881263976607105}, 'large_model': {'SSIM': 0.7370755076408386, 'PSNR (dB)': 13.68035345496387}}


In [13]:
summary = {"orgin":{"SSIM":0,"PSNR (dB)":0},
           "trained":{"SSIM":0,"PSNR (dB)":0},
           "large":{"SSIM":0,"PSNR (dB)":0}}
records = []
for i in range(len(test_dataset)):
    print(f"test epoch {i}/{len(test_dataset)}")
    ret = test_epoch(i)
    summary["orgin"]["SSIM"] += ret["origin_model"]["SSIM"]
    summary["orgin"]["PSNR (dB)"] += ret["origin_model"]["PSNR (dB)"]

    summary["trained"]["SSIM"] += ret["trained_model"]["SSIM"]
    summary["trained"]["PSNR (dB)"] += ret["trained_model"]["PSNR (dB)"]

    summary["large"]["SSIM"] += ret["large_model"]["SSIM"]
    summary["large"]["PSNR (dB)"] += ret["large_model"]["PSNR (dB)"]
    print(json.dumps(ret))
    records.append(ret)

length = len(test_dataset)
summary["orgin"]["SSIM"] /= length
summary["orgin"]["PSNR (dB)"] /= length

summary["trained"]["SSIM"] /= length
summary["trained"]["PSNR (dB)"] /= length

summary["large"]["SSIM"] /= length
summary["large"]["PSNR (dB)"] /= length

print(summary)


test epoch 0/1000


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:02<00:00, 21.36it/s]


{"idx": 0, "prompt": "lay blocks one atop another                       ", "origin_model": {"SSIM": 0.1089048758149147, "PSNR (dB)": 6.271239509054612}, "trained_model": {"SSIM": 0.5569770932197571, "PSNR (dB)": 9.383452193848052}, "large_model": {"SSIM": 0.6934952735900879, "PSNR (dB)": 12.391504223202976}}
test epoch 1/1000


100%|██████████| 50/50 [00:02<00:00, 20.68it/s]


...

{"idx": 997, "prompt": "group blocks into a stack                         ", "origin_model": {"SSIM": 0.023855919018387794, "PSNR (dB)": 4.079657900790984}, "trained_model": {"SSIM": 0.6829248070716858, "PSNR (dB)": 11.154602159760216}, "large_model": {"SSIM": 0.8034573197364807, "PSNR (dB)": 14.53277047546942}}
test epoch 998/1000


100%|██████████| 50/50 [00:02<00:00, 19.78it/s]


{"idx": 998, "prompt": "turn over the blocks                              ", "origin_model": {"SSIM": 0.023146316409111023, "PSNR (dB)": 5.0195226571792615}, "trained_model": {"SSIM": 0.6732616424560547, "PSNR (dB)": 15.765255821120261}, "large_model": {"SSIM": 0.8230829834938049, "PSNR (dB)": 14.718645276291987}}
test epoch 999/1000


100%|██████████| 50/50 [00:02<00:00, 20.20it/s]


{"idx": 999, "prompt": "pass the blocks                                   ", "origin_model": {"SSIM": 0.042841970920562744, "PSNR (dB)": 5.917957378358588}, "trained_model": {"SSIM": 0.7189698815345764, "PSNR (dB)": 12.268441932609885}, "large_model": {"SSIM": 0.6371604800224304, "PSNR (dB)": 9.882318996758519}}
{'orgin': {'SSIM': 0.06047739235777408, 'PSNR (dB)': 5.17495590029061}, 'trained': {'SSIM': 0.678704203903675, 'PSNR (dB)': 11.829665794419757}, 'large': {'SSIM': 0.7227518240213394, 'PSNR (dB)': 12.52209272393689}}


In [14]:
with open("testrecord.json","w+") as f:
    json.dump(records,f)

with open("summary.json","w+") as f:
    json.dump(summary,f)